In [ ]:
import os
import pandas as pd
from psycopg import sql
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import AutoMinorLocator

from swed_17.snow17 import SweDB
from swed_17.zone_db import CBRFCZone

In [ ]:
BASE_DIR = "/perc10/data/cbrfcSnowModel/"

SWE_DB = CBRFCZone("service=swe_db")
SNOW17_DB = SweDB(os.environ.get("S17_DB"))

In [ ]:
# FRGC
# zone_ids = [968, 967, 966]

In [ ]:
# FPTC
zone_ids = [1019, 1018, 1017]

In [ ]:
ZONE_QUERY = """
SELECT cz.gid, cc.ch5_id, cz.segment, cz.zone, cc.description
 FROM cbrfc_zones cz LEFT JOIN cbrfc_ch5id cc ON cz.ch5_id = cc.id
 WHERE cz.gid in ({});
"""
zone_query = sql.SQL(ZONE_QUERY).format(
    sql.SQL(",").join(map(sql.Literal, zone_ids))
)

In [ ]:
with SWE_DB.query(zone_query) as results:
    zones = pd.DataFrame(
        results.fetchall(),
        columns=['ID', 'CH5ID', 'Segment', 'Zone Name', 'Description']
    ).set_index('ID')

In [ ]:
zones

In [ ]:
START_DATE = "2020-10-01"
SWE_QUERY = """
SELECT *
 FROM public.zonal_swe
 WHERE
    cbrfc_zone_id in ({}) AND
    date >= to_date({}, 'YYYY-MM-DD')
"""

In [ ]:
def snow_17_swe_for_zone(zone_id):
    df = SNOW17_DB.for_zone_forecasted(zone_id, from_year=2020)
    df.rename(columns={"SWE (mm)": "Snow-17"}, inplace=True)
    df["Zone Name"] = df["Zone Name"].astype("string")
    # Need to reset index to be able to merge on Date and Zone Name
    df = df.reset_index()
    df["Date"] = df["Date"].dt.tz_localize("UTC")
    return df

In [ ]:
swedb_query = sql.SQL(SWE_QUERY).format(
    sql.SQL(",").join(map(sql.Literal, zone_ids)), START_DATE
)

with SWE_DB.query(swedb_query) as results:
    swe = pd.DataFrame(
        results.fetchall(),
        columns=[
            "Date",
            "Zone Name",
            "iSnobal",
            "SNODAS",
            "UArizona",
            "CU Boulder",
            "ASO",
            "ID",
        ],
    )
swe["Zone Name"] = swe["Zone Name"].astype("string")

In [ ]:
df = pd.merge(
    snow_17_swe_for_zone(zones.iloc[0]["Segment"][0:6]),
    swe,
    on=["Date", "Zone Name"],
    how="inner",
).set_index("Date")
df.sort_values(by='Date', inplace=True)
df.drop("ID", axis=1, inplace=True)
df.drop("Segment", axis=1, inplace=True)

In [ ]:
df

In [ ]:
COLORS = {
    'UF': 'steelblue',
    'MF': 'goldenrod',
    'LF': 'darkorchid',
    'OF': 'seagreen',
}

In [ ]:
plt.figure(dpi=300, figsize=(16, 5))
ax1 = plt.gca()

line_opts = dict(alpha=0.8, zorder=1)
scatter_opts = dict(s=20, zorder=3, edgecolor="black")

for name, group in df.groupby("Zone Name"):
    color_id = name[6:8]
    x_values = group.index
    ax1.plot(
        x_values,
        group["Snow-17"],
        label="",
        color=COLORS[color_id],
        lw=2, zorder=0,
    )
    ax1.plot(
        x_values,
        group["iSnobal"],
        label="",
        color=COLORS[color_id],
        dashes=[6, 1],
        **line_opts
    )
    ax1.plot(
        x_values,
        group["UArizona"],
        label="",
        color=COLORS[color_id],
        # dashes=[6, 4],
        lw=2,
        ls="-.",
        **line_opts
    )
    ax1.plot(
        x_values,
        group["SNODAS"],
        label="",
        lw=2,
        ls=":",
        # dashes=[8, 6],
        color=COLORS[color_id],
        **line_opts
    )
    ax1.scatter(
        x_values,
        group["CU Boulder"],
        label="",
        marker="^",
        color=COLORS[color_id],
        **scatter_opts,
    )
    ax1.scatter(
        x_values,
        group["ASO"],
        label="",
        marker="D",
        color=COLORS[color_id],
        **scatter_opts,
    )

ax1.axhline(-10, lw=2, label="Snow-17", color="grey")
ax1.axhline(-10, dashes=[6, 1], label="iSnobal", color="grey")
ax1.axhline(-10, lw=2, ls="-.", label="UA", color="grey")
ax1.axhline(-10, lw=2, ls=":", label="SNODAS", color="grey")
ax1.scatter(-10, -10, marker="^", label="CUB", color="grey")
ax1.scatter(-10, -10, marker="D", label="ASO", color="grey")

ax1.legend(
    loc='upper right',
    bbox_to_anchor=(1, 1.015),
    ncol=7,
    prop={'size': 10},
    columnspacing=0.5,
    handletextpad=0.5,
    frameon=False,
)
ax1.set_xlim(df.index[0], df.index[-1])
ax1.set_ylim(-7, 700)
ax1.xaxis.set_major_locator(mdates.MonthLocator(interval=6))
ax1.xaxis.set_minor_locator(mdates.MonthLocator(interval=1))
ax1.yaxis.set_minor_locator(AutoMinorLocator())

ax1.set_ylabel("SWE [mm]")
ax1.set_xlabel("Month")
ax1.set_title(
    "Zonal SWE",
    x=0.05, y=0.92, fontweight='bold',
);

In [ ]:
df["CU Boulder"] = df.groupby('Zone Name')["CU Boulder"].transform(
    lambda grp: grp.interpolate(method='nearest')
)

In [ ]:
# Fraser
# date = "2025-04-07"
# Fryingpan
date = "2023-04-12"
peak_swe = df.loc[date]
peak_swe

In [ ]:
fig = plt.figure(dpi=300, figsize=(6, 5))
ax = fig.gca()

for name, group in peak_swe.groupby("Zone Name"):
    annotate_offset = 5
    if name.endswith("LF"):
        color = "darkorchid"
        label = "Lower"
        annotate_offset = -14
    elif name.endswith("MF"):
        color = "goldenrod"
        label = "Middle"
        # annotate_offset = -12
    elif name.endswith("UF"):
        color = "steelblue"
        label = "Upper"
    labels = group.columns.values[1:]
    x = np.arange(0, 2.4, 0.4)
    y = group.iloc[0].values[1:]
    ax.plot(x, y, alpha=0.8, lw=1, color=color, label=label)
    ax.scatter(x, y, marker='.', color=color, label="")
    ax.axhline(y.mean(), ls="--", alpha=0.6, color=color, label="")

    for i, label in enumerate(y):
        ax.annotate(
            int(label.round()),
            (x[i], y[i]),
            textcoords="offset points",
            xytext=(0, annotate_offset),
            ha='center'
        )

ax.axhline(-10, ls="--", label="Mean", color="grey")

ax.legend(
    loc='upper right',
    bbox_to_anchor=(1, 1.015),
    ncol=4,
    prop={'size': 10},
    columnspacing=0.5,
    handletextpad=0.5,
    frameon=False,
)

ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.set_ylim(0, 700)
ax.set_ylabel("SWE [mm]")
ax.yaxis.set_minor_locator(AutoMinorLocator())

ax.set_title(
    date,
    x=0.11, y=0.935,
    fontsize=10, fontweight="bold"
);